# MNIST

http://yann.lecun.com/exdb/mnist/



In [4]:
# Adapted from: https://docs.python.org/3/library/gzip.html

import gzip

with gzip.open('data/train-images-idx3-ubyte.gz', 'rb') as f:
    file_content = f.read()
    


b'\x00\x00\x08\x03'

In [9]:
file_content[0:4]

b'\x00\x00\x08\x03'

In [8]:
int.from_bytes(file_content[0:4], byteorder='big')


2051

In [14]:
int.from_bytes(file_content[16:200], byteorder='big')


1388866522954669608556503841423177209855858011703043733141072446173248093693

In [17]:
# First Image file_content[16:800]
myInt = int.from_bytes(file_content[16:800], 'big')